**Activate `dwh` environment**

And Install google-cloud-bigquery-storage, google-cloud-bigquery packages if not done yet.

The following codes are for data analysis

1. Seller performance (Top sellers by year)
2. Product performance (Top selling product by category by year by city)

In [1]:
#Authenticate the user to access the bigquery API

from google.auth import default
from google.cloud import bigquery

# Authenticate the user
# run `gcloud auth application-default login` in your terminal to authenticate using your auth token

# Set the project ID
project_id = 'premium-node-451703-i2'

# Use the default credentials
credentials, project = default()

# Initialize the BigQuery client
client = bigquery.Client(credentials=credentials, project=project_id)

print(f"Successfully authenticated with project: {project_id}")

Successfully authenticated with project: premium-node-451703-i2


**Distinct value of order_status from orders table for reference** 

0	approved

1	canceled

2	created

3	delivered

4	invoiced

5	processing

6	shipped

7	unavailable

Note: The facts table only contains the orders which are in all status except `canceled`, `created` and `unavailable` as those orders are not confirmed and have no product information attached.



In [7]:
#Fetch information from facts table and put into dataframe
query="select * from premium-node-451703-i2.brazilecom_facts.facts_orders limit 300"
df_res = client.query(query).to_dataframe()
df_res["seller_id"]

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Name: seller_id, Length: 100, dtype: object

In [8]:
df[df['seller_id'] not null]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (241468525.py, line 1)

**Use Case 1: To find the best sellers by year**

In [6]:
# group by seller_id, order_year, seller_city and sum price and freight_value 
df_summary_bysellerbyyearbycity = df_res.groupby(['seller_id', 'order_year','seller_city']).agg({'price': 'sum', 'freight_value': 'sum'}).reset_index()
df_summary_bysellerbyyearbycity.sort_values(by=['order_year', 'seller_city','price'], ascending=[True, False, True], inplace=True)
df_summary_bysellerbyyearbycity.head()

,seller_id,order_year,seller_city,price,freight_value


**Use Case 2: To find best product category by year by city**

In [4]:
df_bestproduct_byyearbycity = df_res.groupby(['product_category_name_english', 'order_year','customer_city']).agg({'price': 'sum', 'freight_value': 'sum'}).reset_index()
df_bestproduct_byyearbycity.sort_values(by=['order_year', 'customer_city','price'], ascending=[True, False, True], inplace=True)
df_bestproduct_byyearbycity.head()

,product_category_name_english,order_year,customer_city,price,freight_value


In [4]:
import pandas as pd

# Read the CSV file from the seeds folder
csv_file_path = 'seeds/geolocation.csv'  # Replace 'your_file_name.csv' with the actual file name
df_csv = pd.read_csv(csv_file_path)

# Display the first few rows of the dataframe
df_csv.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [15]:

# Find distinct geolocation_city and geolocation_zip_code_prefix
distinct_geolocation = df_csv[['geolocation_city']].drop_duplicates()
distinct_geolocation.to_csv('distinct_geolocation.csv', index=False)


In [ ]:
# Calculate the size of each chunk
chunk_size = len(distinct_geolocation) // 10

# Split the DataFrame into 10 smaller DataFrames
split_dataframes = [distinct_geolocation.iloc[i:i + chunk_size] for i in range(0, len(distinct_geolocation), chunk_size)]

# Ensure there are exactly 10 DataFrames (in case of rounding issues)
split_dataframes = split_dataframes[:10]

# Display the first few rows of each DataFrame
for i, df in enumerate(split_dataframes):
    print(f"DataFrame {i + 1}:")
    print(df.to_csv(f"{i+1}.csv"))
    print()

In [ ]:
# Define a mapping dictionary for city names
city_name_mapping = {
    'sao paulo': 'São Paulo',
    'são paulo': 'São Paulo',
    'sao bernardo do campo': 'São Bernardo do Campo',
    'jundiaí': 'Jundiaí',
    'taboão da serra': 'Taboão da Serra',
    # Add more mappings as needed
}

# Tag similar city names with their English equivalents
distinct_geolocation['english_city_name'] = distinct_geolocation['geolocation_city'].map(city_name_mapping).fillna(distinct_geolocation['geolocation_city'])

distinct_geolocation.head()

In [10]:

sao_paulo_cities = distinct_geolocation[distinct_geolocation['geolocation_city'].str.startswith('sao paulo')]
sao_paulo_cities

,geolocation_city
0,sao paulo
737892,sao paulo do potengi
778019,sao paulo de olivenca
989776,sao paulo das missoes


In [1]:
#test if the GCP is still working
import pandas_gbq as pdgbq
import pandas as pd

bq_dataset_id = "brazilecom"
bq_project_id = "premium-node-451703-i2"
data=pd.DataFrame({'date': ['2023-10-01']})
pdgbq.to_gbq(data, f"{bq_dataset_id}.test", project_id=bq_project_id, if_exists="replace")

100%|██████████| 1/1 [00:00<?, ?it/s]


In [5]:
import kagglehub
import os
import pandas as pd
import pandas_gbq as pdgbq

bq_dataset_id = "brazilecom"
bq_project_id = "premium-node-451703-i2"
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce",force_download=True)
for file_name in os.listdir(path):
    full_file_name = os.path.join(path, file_name)
    if os.path.isfile(full_file_name):
        fname=full_file_name.replace("olist_", "")
        fname=fname.replace("_dataset", "")
        fname = os.path.basename(fname)
        fname = fname.replace(".csv","")
        print(f"table name={fname}, fullname={bq_dataset_id}.{fname}, project={bq_project_id}")
        data=pd.read_csv(full_file_name)
        pdgbq.to_gbq(data, f"{bq_dataset_id}.{fname}", project_id=bq_project_id, if_exists="replace")
        break

Resuming download from 37748736 bytes (6968844 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce?dataset_version_number=2 (37748736/44717580) bytes left.


100%|██████████| 42.6M/42.6M [00:32<00:00, 212kB/s] 

Extracting files...


table name=customers, fullname=brazilecom.customers, project=premium-node-451703-i2


100%|██████████| 1/1 [00:00<?, ?it/s]
